## Phase 3: Data Processing & Cleaning

### Why is Data Cleaning Critical?

"Garbage in, garbage out" - Data quality directly impacts model quality.

According to research, data scientists spend 70-80% of time on data cleaning and preparation.

### Common Data Quality Issues

**1. Missing Values**


In [ ]:
import pandas as pd
import numpy as np

# Load data
df = raw_data.copy()

# Check missing values
print(df.isnull().sum())
print(df.isnull().sum() / len(df) * 100)  # Percentage

# Output example:
# transaction_count    0 (0.0%)
# total_spent          0 (0.0%)
# age                  542 (5.4%)
# support_tickets      2104 (21%)
# last_support_date    2104 (21%)


**Strategies for Handling Missing Values:**


In [ ]:
# Strategy 1: Drop rows (only if <5% missing)
df_dropped = df.dropna(subset=['age'])

# Strategy 2: Drop columns (only if >50% missing)
df = df.drop(columns=['last_support_date'])  # 21% missing, might drop

# Strategy 3: Imputation with mean/median (numeric)
df['age'].fillna(df['age'].median(), inplace=True)

# Strategy 4: Imputation with mode (categorical)
df['country'].fillna(df['country'].mode()[0], inplace=True)

# Strategy 5: Forward fill / Backward fill (time series)
df['balance'] = df['balance'].fillna(method='ffill')

# Strategy 6: Advanced imputation (ML-based)
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
df[numeric_cols] = imputer.fit_transform(df[numeric_cols])


**2. Duplicate Records**


In [ ]:
# Check duplicates
print(df.duplicated().sum())  # Exact duplicates

# Check duplicates on specific columns
print(df.duplicated(subset=['customer_id']).sum())

# Remove duplicates (keep first occurrence)
df = df.drop_duplicates(subset=['customer_id'], keep='first')


**3. Data Type Issues**


In [ ]:
# Identify type issues
print(df.dtypes)

# Example: Age column stored as string "25" instead of int
df['age'] = pd.to_numeric(df['age'], errors='coerce')

# Convert dates
df['account_creation_date'] = pd.to_datetime(df['account_creation_date'], 
                                             errors='coerce')

# Convert categorical
df['account_type'] = df['account_type'].astype('category')


**4. Inconsistent/Invalid Values**


In [ ]:
# Check ranges
print(df['age'].describe())

# Example: Age > 150 is unrealistic
df = df[df['age'] <= 120]

# Example: Transaction amount < 0 is invalid
df = df[df['transaction_amount'] > 0]

# Case inconsistency in categorical
df['country'] = df['country'].str.upper()

# Whitespace issues
df['country'] = df['country'].str.strip()


**5. Outliers**


In [ ]:
# Method 1: Statistical (IQR)
Q1 = df['transaction_amount'].quantile(0.25)
Q3 = df['transaction_amount'].quantile(0.75)
IQR = Q3 - Q1
outliers = df[(df['transaction_amount'] < Q1 - 1.5*IQR) | 
              (df['transaction_amount'] > Q3 + 1.5*IQR)]

# Method 2: Z-score
from scipy import stats
z_scores = np.abs(stats.zscore(df['transaction_amount']))
outliers = df[z_scores > 3]

# Method 3: Isolation Forest (ML-based anomaly detection)
from sklearn.ensemble import IsolationForest
iso_forest = IsolationForest(contamination=0.05)
outliers = df[iso_forest.fit_predict(df[['transaction_amount']]) == -1]

# Decision: Keep or Remove? 
# For fraud detection: KEEP (outliers might be fraudulent)
# For customer churn: REMOVE (extreme cases might distort pattern)


### Complete Data Cleaning Pipeline


In [ ]:
def clean_data(df):
    """
    Complete data cleaning pipeline
    """
    # Step 1: Remove duplicates
    df = df.drop_duplicates(subset=['customer_id'], keep='first')
    
    # Step 2: Type conversion
    df['age'] = pd.to_numeric(df['age'], errors='coerce')
    df['account_creation_date'] = pd.to_datetime(df['account_creation_date'])
    
    # Step 3: Handle missing values
    numeric_cols = df.select_dtypes(include=['number']).columns
    categorical_cols = df.select_dtypes(include=['object']).columns
    
    for col in numeric_cols:
        if df[col].isnull().sum() > 0:
            df[col].fillna(df[col].median(), inplace=True)
    
    for col in categorical_cols:
        if df[col].isnull().sum() > 0:
            df[col].fillna('Unknown', inplace=True)
    
    # Step 4: Handle outliers (remove extreme cases)
    df = df[df['age'] <= 120]
    df = df[df['transaction_amount'] > 0]
    
    # Step 5: Standardize formats
    df['country'] = df['country'].str.upper().str.strip()
    
    # Step 6: Log cleaning report
    print(f"Final dataset shape: {df.shape}")
    print(f"Missing values: {df.isnull().sum().sum()}")
    
    return df

cleaned_data = clean_data(raw_data)


### Tools Used in Data Cleaning

| Tool | Purpose |
|------|---------|
| Pandas | Data manipulation, cleaning |
| NumPy | Numerical operations |
| Great Expectations | Data validation & profiling |
| Deequ | Large-scale data quality |
| OpenRefine | Visual data cleaning |
| Apache Spark | Distributed data cleaning |

---
